In [19]:
import tkinter as tk
import random
import requests
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Define URL to scrape
url = "https://en.wikipedia.org/wiki/Y._S._Jagan_Mohan_Reddy"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    raw_text = soup.get_text(separator=" ")
else:
    print("FAILED TO SCRAP THE WEB PAGE")

# Tokenize the text into sentences and words
sent_tokens = nltk.sent_tokenize(raw_text)
word_tokens = nltk.word_tokenize(raw_text)

# Initialize WordNet lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define functions for text normalization and lemmatization
def LemTokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in string.punctuation and token not in stop_words]
    return LemTokens(tokens)

# Define TF-IDF vectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

# Define greeting inputs and responses
GREETING_INPUTS = ("hello", "hi", "hey", "hola", "greetings", "what's up", "howdy", "yo", "hi there", "good day", "morning", "afternoon", "evening", "sup", "yo yo", "hey there", "nice to meet you", "hello there", "what's happening", "how's it going")
GREETING_RESPONSES = ["Hello!", "Hi!", "Hey!", "Hola!", "Greetings!", "What's up?", "Howdy!", "Yo!", "Hi there!", "Good day!", "Good morning!", "Good afternoon!", "Good evening!", "Sup!", "Yo yo!", "Hey there!", "Nice to meet you!", "Hello there!", "Not much, you?", "It's going well, thanks!"]


# Define the greeting function
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Create the main window
root = tk.Tk()
root.title("Chatbot")
root.geometry("400x400")

# Create chat history text widget
chat_history_text = tk.Text(root, height=15, width=50, wrap="word", font=("Arial", 12))
chat_history_text.pack(pady=10)

# Create user input field
user_input_entry = tk.Entry(root, width=50, font=("Arial", 12))
user_input_entry.pack(pady=5)

# Define the response function
# Define the response function
def response(user_response):
    bot_response = ''
    sent_tokens.append(user_response)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if req_tfidf == 0:
        bot_response = "im sorry i ididnt get u"
    else:
        bot_response = sent_tokens[idx]
    return bot_response


# Define the send_message function
def send_message():
    user_input = user_input_entry.get()
    # Insert user input into chat history
    chat_history_text.insert(tk.END, f"You: {user_input}\n")
    # Generate and insert bot response into chat history
    bot_response = response(user_input)
    if greeting(user_input) is not None:
        bot_response = greeting(user_input)
    chat_history_text.insert(tk.END, f"BOT: {bot_response}\n")
    # Scroll to the bottom of the chat history
    chat_history_text.see(tk.END)

    # Clear the user input field
    user_input_entry.delete(0, tk.END)

# Create send button
send_button = tk.Button(root, text="Send", command=send_message, font=("Arial", 12))
send_button.pack(pady=5)

# Configure tags for chat history text
chat_history_text.tag_config("user", foreground="blue")
chat_history_text.tag_config("bot", foreground="red")

# Start the Tkinter event loop
root.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\janap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
